导入我们需要的模块

In [1]:
from sklearn.datasets import make_blobs
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

实例化数据集,可视化数据集

In [ ]:
X,y = make_blobs(n_samples=50,centers=2,random_state=0,cluster_std=0.6)
#散点图
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='rainbow')
ax = plt.gca() #获取当前的子图，如果不存在，则创建新的子图

# 去掉横纵坐标轴的刻度
plt.xticks([])
plt.yticks([])
plt.show()


制作网格, 画决策边界

In [ ]:
#获取平面上两条坐标轴的最大值和最小值
xlim = ax.get_xlim()
ylim = ax.get_ylim()
 
#在最大值和最小值之间形成30个规律的数据
axisx = np.linspace(xlim[0],xlim[1],30)
axisy = np.linspace(ylim[0],ylim[1],30)

#使用meshgrid函数将两个一维向量转换为特征矩阵
#核心是将两个特征向量广播，以便获取y.shape * x.shape这么多个坐标点的横坐标和纵坐标
axisx,axisy = np.meshgrid(axisx,axisy)

#其中ravel()是降维函数，vstack能够将多个结构一致的一维数组按行堆叠起来
#xy就是已经形成的网格，它是遍布在整个画布上的密集的点
xy = np.vstack([axisx.ravel(),axisy.ravel()]).T

plt.scatter(xy[:,0],xy[:,1],s=1,cmap='rainbow')

In [ ]:
#理解函数meshgrid和vstack的作用
a = np.array([1,2,3])
b = np.array([7,8])

v1,v2 = np.meshgrid(a,b)
print(v1)
print(v2)

v = np.vstack([v1.ravel(), v2.ravel()]).T
print(v1.ravel())
print(v2.ravel())
print(v)


In [ ]:
#建模,通过fit计算出对应的决策边界
clf =SVC(kernel='linear').fit(X,y)
Z = clf.decision_function(xy).reshape(axisx.shape)
# 重要接口decision_function返回每个输入的样本所对应的到决策边界的距离
# 然后再将这个距离转换为axisx的结构,这是由于画图的函数contour要求z的结构必须与x和y保持一致

#首先要有散点图
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap="rainbow")
ax = plt.gca() #获取当前的子图，如果不存在，则创建新的子图
#画决策边界和平行于决策边界的超平面
ax.contour(axisx
           ,axisy
           ,Z
           ,colors='k'
           ,levels=[-1,0,1] #画三条等高线,分别是z为-1,0和1的三条线
           ,alpha=0.5
           ,linestyles=['--','-','--'])
ax.set_xlim(xlim)
ax.set_ylim(ylim)


In [ ]:
#Z的本质是输入的样本到决策边界的距离，而contour函数中的level其实是输入了这个距离 
# #让我们用一个点来试试看
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap="rainbow") 
plt.scatter(X[10,0],X[10,1],c="black",s=50,cmap="rainbow")

print(clf.decision_function(X[10].reshape(1,2)))

In [ ]:
clf.decision_function(X[10].reshape(1,2))
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap="rainbow")
ax = plt.gca()
ax.contour(axisx,axisy,Z
            ,colors="k"
            ,levels=[-3.33917354]
            ,alpha=0.5
            ,linestyles=["--"])

In [ ]:
#将上述过程包装成函数:
def plot_svc_decision_function(model,ax=None):
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    x = np.linspace(xlim[0],xlim[1],30)
    y = np.linspace(ylim[0],ylim[1],30)
    Y,X = np.meshgrid(y,x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    Z = model.decision_function(xy).reshape(X.shape)

    # x = np.arange(xlim[0],xlim[1], 1)
    # y = np.arange(ylim[0],ylim[1], 1)
    # Y,X = np.meshgrid(y, x)
    # xy = np.c_[X.ravel(), Y.ravel()]
    # Z = clf.predict(xy).reshape(X.shape)
    
    ax.contour(X, Y, Z,colors="k",levels=[-1,0,1],alpha=0.5,linestyles=["--","-","--"])
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    

In [ ]:
#整个绘图过程可以写作:
clf = SVC(kernel = "linear").fit(X,y) 
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap="rainbow") 
plot_svc_decision_function(clf)


In [ ]:
#探索建好的模型
clf.predict(X) #根据决策边界，对X中的样本进行分类，返回的结构为n_samples
clf.score(X,y) #返回给定测试数据和标签的平均准确度
clf.support_vectors_ #返回支持向量
clf.n_support_ #返回每个类中支持向量的个数


推广到非线性情况
定义一个由x计算出来的新维度r


In [ ]:
from sklearn.datasets import make_circles
X,y = make_circles(100,factor=0.1,noise=.1)
X.shape
y.shape
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='rainbow')
plt.show()

In [ ]:
# 试试看用我们已经定义的函数来划分这个数据的决策边界

clf = SVC(kernel = 'linear').fit(X,y)
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='rainbow')
plot_svc_decision_function(clf)


In [ ]:
#为非线性数据增加纬度并绘制3D图像
#定义一个由x计算出来的新维度r

r = np.exp(-(X**2).sum(1))

rlim = np.linspace(min(r),max(r),0.2)

from mpl_toolkits import mplot3d

# 定义一个绘制三维图像的函数
# elev表示上下旋转的角度
# azim表示平行旋转的角度
def plot_3D(elev=30,azim=30,X=X,y=y):
    ax = plt.subplot(projection="3d")
    ax.scatter3D(X[:,0],X[:,1],r,c=y,s=50,cmap='rainbow')
    ax.view_init(elev=elev,azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')
    plt.show()
    
plot_3D()

from ipywidgets import interact,fixed
interact(plot_3D,evel=[0.30],azip=(-180.180),X=fixed(X),y=fixed(y))
plt.show()

此时我们的数据在三维空间中，我们的超平面就是一个二维平面。明显我们可以用一个平面将两类数据隔开，这个平面就是我们的决策边界了。我们刚才做的计算r，并将r作为数据的第三维度来将数据升维的过程，被称为“核变换”，即是将数据投影到高维空间中，以寻找能够将数据完美分割的超平面，即是说寻找能够让数据线性可分的高维空间。为了详细解释这个过程，我们需要引入SVM中的核心概念：核函数。

# 非线性SVM与核函数
## SVC在非线性数据上的推广
为了能够找出非线性数据的线性决策边界，我们需要将数据从原始的空间$x$投射到新空间${\Phi}(x)$中。${\Phi}(x)$是一个映射函数，它代表了某种非线性的变换，如同我们之前所做过的使用r来升维一样，这种非线性变换看起来是一种非常有效的方式。使用这种变换，线性SVM的原理可以被很容易推广到非线性情况下，其推导过程和逻辑都与线性SVM 一模一样，只不过在定义决策边界之前，我们必须先对数据进行升维度，即将原始的x转换成${\Phi}(x)$。   
如此，非线性SVM的损失函数的初始形态为：
$$\min_{w,b}\frac{||ω||^2}{2}$$
$$s. \, t. \ y_i(ω\cdot{\Phi(x_i)} + b) \geq 1,i=1,2,...,N$$
同理，非线性SVM的拉格朗日函数和拉格朗日对偶函数也可得：
$$L(w,b,α) = \frac{1}{2}||w||^2 - \sum\limits_{i=1}^{m}α_i[y_i(w\cdot{\Phi(x_i)} + b) - 1] \;  (α_i \geq 0)$$
$$L_d=\sum\limits_{i=1}^{m}\alpha_i  - \frac{1}{2}\sum\limits_{i=1,j=1}^{m}\alpha_i\alpha_jy_iy_j{\Phi(x_i)}{\Phi(x_j)}$$
使用同样的推导方式，让拉格朗日函数满足KKT条件，并在拉格朗日函数上对每个参数求导，经过和线性SVM相同的变换后，就可以得到拉格朗日对偶函数。同样使用梯度下降或SMO等方式对$\alpha$进行求解，最后可以求得决策边界，并得到最终的决策函数：
$$f(x_{test}) = sign(w^{*} \cdot \Phi(x_{test}) + b^{*}) \, = \, sign(\sum\limits_{i=1}^Na_iy_i\Phi(x_i) \cdot \Phi(x_{test}) + b^{*})$$
## 重要参数kernel
这种变换非常巧妙，但也带有一些实现问题。 首先，我们可能不清楚应该什么样的数据应该使用什么类型的映射函数来确保可以在变换空间中找出线性决策边界。极端情况下，数据可能会被映射到无限维度的空间中，这种高维空间可能不是那么友好，维度越多，推导和计算的难度都会随之暴增。其次，即使已知适当的映射函数，我们想要计算类似于$\Phi(x_i) \cdot \Phi(x_{test})$这样的点积，计算量可能会无比巨大，要找出超平面所付出的代价是非常昂贵的。

#### 关键概念：核函数
>而解决这些问题的数学方式，叫做“核技巧”(Kernel Trick)，是一种能够使用数据原始空间中的向量计算来表示升维后的空间中的点积结果的数学方式。具体表现为$K(u,v)=\Phi(u)\cdot\Phi(v)$。而这个原始空间中的点积函数$K(u,v)$，就被叫做“核函数”(Kernel Function)。

核函数能够帮助我们解决三个问题：
第一，有了核函数之后，我们无需去担心 究竟应该是什么样，因为非线性SVM中的核函数都是正定核函数
(positive deﬁnite kernel functions)，他们都满足美世定律(Mercer's theorem)，确保了高维空间中任意两个向量 的点积一定可以被低维空间中的这两个向量的某种计算来表示（多数时候是点积的某种变换）。   
第二，使用核函数计算低维度中的向量关系比计算原本的$\Phi(x_i) \cdot \Phi(x_{test})$要简单太多了。   
第三，因为计算是在原始空间中进行，所以避免了维度诅咒的问题。   
选用不同的核函数，就可以解决不同数据分布下的寻找超平面问题。在SVC中，这个功能由参数“kernel”和一系列 与核函数相关的参数来进行控制。之前的代码中我们一直使用这个参数并输入"linear"，但却没有给大家详细讲
解，也是因为如果不先理解核函数本身，很难说明这个参数到底在做什么。参数“kernel"在sklearn中可选以下几种选项：

<table>
    <tbody>
        <tr class="firstRow">
            <td width="100" valign="top" align="left">
                输入
            </td>
            <td width="100" valign="top" align="left">
                含义
            </td>
            <td width="167" valign="top" align="left">
                解决问题
            </td>
            <td width="200" valign="top" align="left">
                核函数的表达式<br/>
            </td>
            <td width="167" valign="top" align="left">
                参数gamma
            </td>
            <td width="167" valign="top" align="left">
                参数degree
            </td>
            <td width="167" valign="top" align="left">
                参数coef0
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                linear<br/>
            </td>
            <td width="100" valign="top" align="left">
                线性核<br/>
            </td>
            <td width="167" valign="top" align="left">
                线性
            </td>
            <td width="267" valign="top" align="left">
                $K(x,y)=x^Ty=x\cdot{y}$
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                poly
            </td>
            <td width="100" valign="top" align="left">
                多项式核
            </td>
            <td width="167" valign="top" align="left">
                偏线性
            </td>
            <td width="267" valign="top" align="left">
                $K(x,y)=(\gamma(x\cdot{y})+r)^d$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                sigmoid<br/>
            </td>
            <td width="100" valign="top" align="left">
                双曲正切核
            </td>
            <td width="167" valign="top" align="left">
                非线性
            </td>
            <td width="267" valign="top">
                $K(x,y)=tanh(\gamma(x\cdot{y})+r)$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                rbf
            </td>
            <td width="100" valign="top" align="left">
                高斯径向基
            </td>
            <td width="167" valign="top" align="left">
                偏非线性
            </td>
            <td width="267" valign="top">
                            $K(x,y)=e^{-\gamma||x-y||^2},\gamma>0$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
        </tr>
    </tbody>
</table>

可以看出，除了选项"linear"之外，其他核函数都可以处理非线性问题。多项式核函数有次数d，当d为1的时候它就是再处理线性问题，当d为更高次项的时候它就是在处理非线性问题。我们之前画图时使用的是选项“linear"，自然不能处理环形数据这样非线性的状况。而刚才我们使用的计算r的方法，其实是高斯径向基核函数所对应的功能，在参数”kernel“中输入”rbf“就可以使用这种核函数。我们来看看模型找出的决策边界时什么样：

In [ ]:
# 高斯径向基核rbf
clf = SVC(kernel = 'rbf',gamma='auto').fit(X,y)
plt.scatter(X[:,0],X[:,1],c=y,s=50,cmap='rainbow')
plot_svc_decision_function(clf)



可以看到，决策边界被完美地找了出来。
## 探索核函数在不同数据集上的表现
除了"linear"以外的核函数都能够处理非线性情况，那究竟什么时候选择哪一个核函数呢？遗憾的是，关于核函数 在不同数据集上的研究甚少，谷歌学术上的论文中也没有几篇是研究核函数在SVM中的运用的，更多的是关于核函 数在深度学习，神经网络中如何使用。在sklearn中，也没有提供任何关于如何选取核函数的信息。   
但无论如何，我们还是可以通过在不同的核函数中循环去找寻最佳的核函数来对核函数进行一个选取。接下来我们 就通过一个例子，来探索一下不同数据集上核函数的表现。我们现在有一系列线性或非线性可分的数据，我们希望 通过绘制SVC在不同核函数下的决策边界并计算SVC在不同核函数下分类准确率来观察核函数的效用。   
1. 导入所需要的库和模块

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import svm
from sklearn.datasets import make_circles, make_moons, make_blobs,make_classification

2. 创建数据集，定义核函数的选择

In [ ]:
n_samples = 100
datasets = [
 make_moons(n_samples=n_samples, noise=0.2, random_state=0),
 make_circles(n_samples=n_samples, noise=0.2, factor=0.5, random_state=1),
 make_blobs(n_samples=n_samples, centers=2, random_state=5),#分簇的数据集
 make_classification(n_samples=n_samples,n_features = 2,n_informative=2,n_redundant=0, random_state=5)
    #n_features：特征数，n_informative：带信息的特征数，n_redundant：不带信息的特征数
 ]
#四个数据集分别是什么样子呢？ 
for X,Y in datasets:
    plt.figure(figsize=(5,4))
    plt.scatter(X[:,0],X[:,1],c=Y,s=50,cmap="rainbow")

3. 循环画出每个图

In [ ]:
Kernel = ["linear","poly","rbf","sigmoid"]


[*enumerate(datasets)] == list(enumerate(datasets))#  enumerate、map、zip都可以这样展开
# index，(X,Y) = [(索引, array([特矩阵征X],[标签Y]))]
# 构建子图
nrows=len(datasets)
ncols=len(Kernel) + 1
fig, axes = plt.subplots(nrows, ncols,figsize=(20,16))
#开始进行子图循环
#第一层循环：在不同的数据集中循环 
for ds_cnt, (X,Y) in enumerate(datasets):

    #在图像中的第一列，放置原数据的分布
    ax = axes[ds_cnt, 0] 
    if ds_cnt == 0:
        ax.set_title("Input data") 
    ax.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.cm.Paired,edgecolors='k') 
    ax.set_xticks(()) 
    ax.set_yticks(())

    #第二层循环：在不同的核函数中循环 
    #从图像的第二列开始，一个个填充分类结果
    for est_idx, kernel in enumerate(Kernel):

        #定义子图位置 
        ax = axes[ds_cnt, est_idx + 1]

        #建模 
        clf = svm.SVC(kernel=kernel, gamma=2).fit(X, Y)
        score = clf.score(X, Y)

        #绘制图像本身分布的散点图 
        ax.scatter(X[:, 0], X[:, 1], c=Y ,zorder=10 ,cmap=plt.cm.Paired,edgecolors='k') 
        #绘制支持向量 
        ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=50, facecolors='none', zorder=10, edgecolors='k')

        #绘制决策边界 
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5 
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

        #np.mgrid，合并了我们之前使用的np.linspace和np.meshgrid的用法
        #一次性使用最大值和最小值来生成网格 
        #表示为[起始值：结束值：步长] 
        #如果步长是复数，则其整数部分就是起始值和结束值之间创建的点的数量，并且结束值被包含在内

        XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j] 
        #np.c_，类似于np.vstack的功能 
        Z = clf.decision_function(np.c_[XX.ravel(), YY.ravel()]).reshape(XX.shape) 
        #填充等高线不同区域的颜色 
        ax.pcolormesh(XX, YY, Z > 0, cmap=plt.cm.Paired) 
        #绘制等高线 
        ax.contour(XX, YY, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], levels=[-1, 0, 1])

        #设定坐标轴为不显示 
        ax.set_xticks(()) 
        ax.set_yticks(())

        #将标题放在第一行的顶上 
        if ds_cnt == 0:
            ax.set_title(kernel)

        #为每张图添加分类的分数 
        ax.text(0.95, 0.06
                , ('%.2f' % score).lstrip('0')
                , size=15 
                , bbox=dict(boxstyle='round', alpha=0.8, facecolor='white') #为分数添加一个白色的格子作为底色 
                , transform=ax.transAxes  #确定文字所对应的坐标轴，就是ax子图的坐标轴本身 
                , horizontalalignment='right' #位于坐标轴的什么方向 
               )

plt.tight_layout() 
plt.show()

可以观察到，线性核函数和多项式核函数在非线性数据上表现会浮动，如果数据相对线性可分，则表现不错，如果 是像环形数据那样彻底不可分的，则表现糟糕。在线性数据集上，线性核函数和多项式核函数即便有扰动项也可以 表现不错，可见多项式核函数是虽然也可以处理非线性情况，但更偏向于线性的功能。  
Sigmoid核函数就比较尴尬了，它在非线性数据上强于两个线性核函数，但效果明显不如rbf，它在线性数据上完全 比不上线性的核函数们，对扰动项的抵抗也比较弱，所以它功能比较弱小，很少被用到。   
rbf，高斯径向基核函数基本在任何数据集上都表现不错，属于比较万能的核函数。一般的经验是，无论如何先试试看高斯径向基核函数，它适用于核转换到很高的空间的情况，在各种情况下往往效果都很不错，如果rbf效果不好，那我们再试试看其他的核函数。另外，多项式核函数多被用于图像处理之中。
## 探索核函数的优势和缺陷
看起来，除了Sigmoid核函数，其他核函数效果都还不错。但其实rbf和poly都有自己的弊端，我们使用乳腺癌数据 集作为例子来展示一下：

In [ ]:
from sklearn.datasets import load_breast_cancer 
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt 
import numpy as np 
from time import time 
import datetime

data = load_breast_cancer() 
X = data.data 
y = data.target

X.shape 
np.unique(y)

plt.scatter(X[:,0],X[:,1],c=y) 
plt.show()

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=420)

Kernel = ["linear","poly","rbf","sigmoid"]

for kernel in Kernel:

    time0 = time() 
    clf= SVC(kernel = kernel 
             , gamma="auto"
             , degree = 1 
             , cache_size=5000
            ).fit(Xtrain,Ytrain) 
    print("The accuracy under kernel %s is %f" % (kernel,clf.score(Xtest,Ytest))) 
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))

我们可以有两个发现。首先，乳腺癌数据集是一个线性数据集，线性核函数跑出来的效果很好。rbf和sigmoid两个 擅长非线性的数据从效果上来看完全不可用。其次，线性核函数的运行速度远远不如非线性的两个核函数。  
因为数据是线性的，当我们把degree参数调整为1后，多项式核函数的运行速度立刻加快了，并且精度也提升到了接近线性核函数的水平，符合我们的理解。但是，我们之前说过，rbf在线性数据上也可以表现得非常好，那在这里，为什么跑出来的结果如此糟糕呢？   
其实，这里真正的问题是数据的量纲问题。回忆一下我们如何求解决策边界，如何判断点是否在决策边界的一边？ 是靠计算”距离“，虽然我们不能说SVM是完全的距离类模型，但是它严重受到数据量纲的影响。让我们来探索一下乳腺癌数据集的量纲：

In [ ]:
import pandas as pd 
data = pd.DataFrame(X) 
data.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.99]).T

很明显数据存在严重的量纲不一的问题。我们来使用数据预处理中的标准化的类，对数据进行标准化：

In [2]:
from sklearn.preprocessing import StandardScaler 
X = StandardScaler().fit_transform(X) 
data = pd.DataFrame(X) 
data.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.99]).T

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X,y,test_size=0.3,random_state=420)

Kernel = ["linear","poly","rbf","sigmoid"]

for kernel in Kernel:

    time0 = time() 
    clf= SVC(kernel = kernel 
             , gamma="auto"
             , degree = 1 
             , cache_size=5000
            ).fit(Xtrain,Ytrain) 
    print("The accuracy under kernel %s is %f" % (kernel,clf.score(Xtest,Ytest))) 
    print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))

量纲统一之后，可以观察到，所有核函数的运算时间都大大地减少了，尤其是对于线性核来说，而多项式核函数居 然变成了计算最快的。其次，rbf表现出了非常优秀的结果。经过我们的探索，我们可以得到的结论是：   

1. 线性核，尤其是多项式核函数在高次项时计算非常缓慢

2. rbf和多项式核函数都不擅长处理量纲不统一的数据集

幸运的是，这两个缺点都可以由数据标准化来解决。因此，SVM执行之前，非常推荐先进行数据的标准化！到了这一步，我们是否已经完成建模了呢？虽然线性核函数的效果是最好的，但它是没有核函数相关参数可以调整的，rbf和多项式却还有着可以调整的相关参数，接下来我们就来看看这些参数。

## 选取与核函数相关的参数：degree & gamma & coef0

<table>
    <tbody>
        <tr class="firstRow">
            <td width="100" valign="top" align="left">
                输入
            </td>
            <td width="100" valign="top" align="left">
                含义
            </td>
            <td width="167" valign="top" align="left">
                解决问题
            </td>
            <td width="200" valign="top" align="left">
                核函数的表达式<br/>
            </td>
            <td width="167" valign="top" align="left">
                参数gamma
            </td>
            <td width="167" valign="top" align="left">
                参数degree
            </td>
            <td width="167" valign="top" align="left">
                参数coef0
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                linear<br/>
            </td>
            <td width="100" valign="top" align="left">
                线性核<br/>
            </td>
            <td width="167" valign="top" align="left">
                线性
            </td>
            <td width="267" valign="top" align="left">
                $K(x,y)=x^Ty=x\cdot{y}$
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                poly
            </td>
            <td width="100" valign="top" align="left">
                多项式核
            </td>
            <td width="167" valign="top" align="left">
                偏线性
            </td>
            <td width="267" valign="top" align="left">
                $K(x,y)=(\gamma(x\cdot{y})+r)^d$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                sigmoid<br/>
            </td>
            <td width="100" valign="top" align="left">
                双曲正切核
            </td>
            <td width="167" valign="top" align="left">
                非线性
            </td>
            <td width="267" valign="top">
                $K(x,y)=tanh(\gamma(x\cdot{y})+r)$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
        </tr>
        <tr>
            <td width="100" valign="top" align="left">
                rbf
            </td>
            <td width="100" valign="top" align="left">
                高斯径向基
            </td>
            <td width="167" valign="top" align="left">
                偏非线性
            </td>
            <td width="267" valign="top">
                            $K(x,y)=e^{-\gamma||x-y||^2},\gamma>0$
            </td>
            <td width="167" valign="top" align="left">
                yes
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
            <td width="167" valign="top" align="left">
                no
            </td>
        </tr>
    </tbody>
</table>

在知道如何选取核函数后，我们还要观察一下除了kernel之外的核函数相关的参数。对于线性核函数，"kernel"是唯一能够影响它的参数，但是对于其他三种非线性核函数，他们还受到参数gamma，degree以及coef0的影响。 参数gamma就是表达式中的$\gamma$，degree就是多项式核函数的次数$d$，参数coef0就是常数项$r$。其中，高斯径向基核函数受到gamma的影响，而多项式核函数受到全部三个参数的影响。

<table>
    <tbody>
        <tr class="firstRow">
            <td width="98" valign="top" style="word-break: break-all;text:bold;">
                <p>
                    <strong>参数</strong>&nbsp; &nbsp;
                </p>
            </td>
            <td width="1180" valign="top" style="word-break: break-all;text:bold;">
                <p>
                    <strong>含义</strong>&nbsp; &nbsp;
                </p>
            </td>
        </tr>
        <tr>
            <td width="98" valign="top" style="word-break: break-all;">
                <p>
                    degree&nbsp; &nbsp;
                </p>
            </td>
            <td width="1180" valign="top" style="word-break: break-all;">
                <p>
                    整数，可不填，默认3&nbsp; &nbsp;
                </p>
                <p>
                    多项式核函数的次数（&#39;poly&#39;），如果核函数没有选择&quot;poly&quot;，这个参数会被忽略
                </p>
            </td>
        </tr>
        <tr>
            <td width="98" valign="top" style="word-break: break-all;">
                <p>
                    gamma&nbsp; &nbsp;
                </p>
            </td>
            <td width="1180" valign="top" style="word-break: break-all;">
                <p>
                    浮点数，可不填，默认“auto&quot;&nbsp; &nbsp;
                </p>
                <p>
                    核函数的系数，仅在参数Kernel的选项为”rbf&quot;,&quot;poly&quot;和&quot;sigmoid”的时候有效&nbsp; &nbsp;&nbsp;
                </p>
                <p>
                    输入“auto&quot;，自动使用1/(n_features)作为gamma的取值&nbsp; &nbsp;&nbsp;
                </p>
                <p>
                    输入&quot;scale&quot;，则使用1/(n_features * X.std())作为gamma的取值&nbsp; &nbsp;&nbsp;
                </p>
                <p>
                    输入&quot;auto_deprecated&quot;，则表示没有传递明确的gamma值（不推荐使用）
                </p>
            </td>
        </tr>
        <tr>
            <td width="98" valign="top" style="word-break: break-all;">
                <p>
                    coef0&nbsp; &nbsp;
                </p>
            </td>
            <td width="1180" valign="top" style="word-break: break-all;">
                <p>
                    浮点数，可不填，默认=0.0&nbsp; &nbsp;
                </p>
                <p>
                    核函数中的常数项，它只在参数kernel为&#39;poly&#39;和&#39;sigmoid&#39;的时候有效。
                </p>
            </td>
        </tr>
    </tbody>
</table>

但从核函数的公式来看，我们其实很难去界定具体每个参数如何影响了SVM的表现。当gamma的符号变化，或者 degree的大小变化时，核函数本身甚至都不是永远单调的。所以如果我们想要彻底地理解这三个参数，我们要先推 导出它们如何影响核函数地变化，再找出核函数的变化如何影响了我们的预测函数（可能改变我们的核变化所在的 维度），再判断出决策边界随着预测函数的改变发生了怎样的变化。无论是从数学的角度来说还是从实践的角度来 说，这个过程太复杂也太低效。所以，我们往往避免去真正探究这些参数如何影响了我们的核函数，而直接使用学 习曲线或者网格搜索来帮助我们查找最佳的参数组合。   

对于高斯径向基核函数，调整gamma的方式其实比较容易，那就是画学习曲线。我们来试试看高斯径向基核函数 rbf的参数gamma在乳腺癌数据集上的表现：

In [ ]:
score = []
gamma_range = np.logspace(-10, 1, 50) #开始点和结束点是10的幂,返回在对数刻度上均匀间隔的数字
# print(gamma_range)
for i in gamma_range:
    clf = SVC(kernel="rbf",gamma = i,cache_size=5000).fit(Xtrain,Ytrain)
    score.append(clf.score(Xtest,Ytest))
    
print(max(score), gamma_range[score.index(max(score))])
plt.plot(gamma_range,score)
plt.show()

通过学习曲线，很容就找出了rbf的最佳gamma值。但我们观察到，这其实与线性核函数的准确率一模一样之前的 准确率。我们可以多次调整gamma_range来观察结果，可以发现97.6608应该是rbf核函数的极限了。   

但对于多项式核函数来说，一切就没有那么容易了，因为三个参数共同作用在一个数学公式上影响它的效果，因此 我们往往使用网格搜索来共同调整三个对多项式核函数有影响的参数。依然使用乳腺癌数据集。

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit #用于支持带交叉验证的网格搜索
from sklearn.model_selection import GridSearchCV #带交叉验证的网格搜索
 
time0 = time()
 
gamma_range = np.logspace(-10,1,20)
coef0_range = np.linspace(0,5,10)
 
param_grid = dict(gamma = gamma_range ,coef0 = coef0_range)

cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=420) #将数据分为5份，5份数据中测试集占30%
gsearch = GridSearchCV(SVC(kernel = "poly",degree=1,cache_size=5000)
                        ,param_grid=param_grid
                        ,cv=cv)
gsearch.fit(X, y)
 
print("The best parameters are %s with a score of %0.5f" % (gsearch.best_params_, gsearch.best_score_))
print(datetime.datetime.fromtimestamp(time()-time0).strftime("%M:%S:%f"))

可以发现，网格搜索为我们返回了参数coef0=0，gamma=0.18329807108324375，但整体的分数是0.96959，虽然比调参前略有提高，但依然没有超过线性核函数核rbf的结果。可见，如果最初选择核函数的时候，你就发现多项式的结果不如rbf和线性核函数，那就不要挣扎了，试试看调整rbf或者直接使用线性。

# 硬间隔与软间隔：重要参数C
## SVM在软间隔数据上的推广
到这里，我们已经了解了线性SVC的基本原理，以及SVM如何被推广到非线性情况下，还了解了核函数的选择和应用。但实际上，我们依然没有完全了解sklearn当中的SVM用于二分类的全貌。我们之前在理论推导中使用的数据都有一个特点，那就是他们或是完全线性可分，或者是非线性可分的数据。在我们对比核函数时，实际上用到了一种不同的数据，那就是不完全线性可分的数据集。比如说如下数据集：
![svm8](../images/svm8.png)
这个数据集和我们最开始介绍SVM如何工作的时候的数据集一模一样，除了多了P和Q两个点。我们注意到，虽然决策边界$B_1$的间 已经非常宽了，然而点P和Q依然被分错了类别，相反，边际比较小的$B_2$却正确地分出了点P和 Q的类别。这里并不是说$B_2$此时此刻就是一条更好的边界了，与之前的论述中一致，如果我们引入更多的训练数 据，或引入测试数据，$B_1$更加宽敞的边界可以帮助它又更好的表现。但是，和之前不一样，现在即便是让边际最大的决策边界$B_1$的训练误差也不可能为0了。此时，我们就需要引入“软间隔”的概念：    
**关键概念：硬间隔与软间隔**
>当两组数据是完全线性可分，我们可以找出一个决策边界使得训练集上的分类误差为0，这两种数据就被称为 是存在”硬间隔“的。当两组数据几乎是完全线性可分的，但决策边界在训练集上存在较小的训练误差，这两种 数据就被称为是存在”软间隔“。

我们可以通过调整我们对决策边界的定义，将硬间隔时得出的数学结论推广到软间隔的情况上，让决策边界能够忍受一小部分训练误差。这个时候，我们的决策边界就不是单纯地寻求最大边际了，因为对于软间隔地数据来说，边际越大被分错的样本也就会越多，因此我们需要找出一个”最大边际“与”被分错的样本数量“之间的平衡。
![svm9](../images/svm9.png)
看上图，原始的决策边界$w\cdot x_i +b = 0$，原本的平行于决策边界的两个虚线超平面$w\cdot x_i +b = 1$和$w\cdot x_i +b = -1$ 都依然有效。我们的原始判别函数为：
$$y_i(w\cdot x_i +b) \geq 1 $$
不过，这些超平面现在无法让数据上的训练误差等于0了，因为此时存在了一个混杂在红色点中的紫色点$x_p$。于是，我们需要放松我们原始判别函数中的不等条件，来让决策边界能够适用于我们的异常点，于是我们引入松弛系数$\xi_i \geq 0$来帮助我们优化原始的判别函数：
$$y_i(w\cdot x_i +b) \geq 1- \xi_i$$

可以看得出，这其实是将原本的虚线超平面向图像的上方和下方平移，其符号的处理方式和我们原本讲解过的把符号放入$w$是一模一样的方式。松弛系数其实很好理解，来看上面的图像。位于红色点附近的紫色点$X_p$在原本的判别函数中必定会被分为红色，所以一定会被判断错。现在我们作一条与决策边界平行，但是过点$x_p$的直线$w\cdot x_i +b = 1- \xi_i$（图中的蓝色虚线）。这条直线是由$w\cdot x_i +b = 1$平移得到，所以两条直线在纵坐标上的差异就是$\xi$（竖直的黑色箭头）。而点$x_p$到$w\cdot x_i +b = 1$的距离就可以表示为$\frac{{\xi}\cdot{w}}{||w||}$，即$\xi$在$w$方向上的投影。由于单位向量是固定的，所以$\xi$可以作为点$x_p$在原始的决策边界上的分类错误的程度的表示，隔得越远，分得越错。但注意，$\xi$并不是点到决策超平面的距离本身。  
   
不难注意到，我们让 作为我们的新决策超平面，是由一定的问题的，虽然我们把异常的紫色点 分类正确了，但我们同时也分错了一系列红色的点。所以我们必须在我们求解最大边际的损失函数中加上一个惩罚 项，用来惩罚我们具有巨大松弛系数的决策超平面。我们的拉格朗日函数，拉格朗日对偶函数，也因此都被松弛系 数改变。现在，我们的损失函数为：

$$\min_{w,b,\xi}\;\; \frac{1}{2}||w||^2 +C\sum\limits_{i=1}^{m}\xi_i$$
$$s.t.  \;\; y_i(w\cdot\phi(x_i) + b)  \geq 1 - \xi_i \;\;(i =1,2,...m)$$
$$\xi_i \geq 0 \;\;(i =1,2,...m)$$

这里,C>0为惩罚参数，可以理解为我们一般回归和分类问题正则化时候的参数。C越大，对误分类的惩罚越大，C越小，对误分类的惩罚越小。   
也就是说，我们希望$\frac{||w||^2}{2}$尽量小，误分类的点尽可能的少。C是协调两者关系的正则化惩罚系数。在实际应用中，需要调参来选择。   
这个目标函数的优化和前面的线性可分SVM的优化方式类似，我们下面就来看看怎么对线性分类SVM的软间隔最大化来进行学习优化。    
和线性可分SVM的优化方式类似，我们首先将软间隔最大化的约束问题用拉格朗日函数转化为无约束问题如下：
$$L(w,b,\xi,\alpha,\mu) = \frac{1}{2}||w||^2 +C\sum\limits_{i=1}^{m}\xi_i - \sum\limits_{i=1}^{m}\alpha_i[y_i(w^T\Phi(x_i) + b) - 1 + \xi_i] - \sum\limits_{i=1}^{m}\mu_i\xi_i$$
需要满足的KKT条件为:
$$\frac{\partial L}{\partial w} = 0 \;\Rightarrow w = \sum\limits_{i=1}^{m}\alpha_iy_ix_i$$
$$\frac{\partial L}{\partial b} = 0 \;\Rightarrow \sum\limits_{i=1}^{m}\alpha_iy_i = 0$$
$$\frac{\partial L}{\partial \xi} = 0 \;\Rightarrow C- \alpha_i - \mu_i = 0$$
$$\alpha_i \geq 0,\xi_i \geq 0,\mu_i \geq 0$$
$$\alpha_i[y_i(w^T\Phi(x_i) + b) - 1 + \xi_i]=0$$
$$\mu_i\xi_i = 0$$
利用上面的三个式子去消除w和b

$$\begin{align} 
L(w,b,\xi,\alpha,\mu) & = \frac{1}{2}||w||_2^2 +C\sum\limits_{i=1}^{m}\xi_i - \sum\limits_{i=1}^{m}\alpha_i[y_i(w^Tx_i + b) - 1 + \xi_i] - \sum\limits_{i=1}^{m}\mu_i\xi_i 　\tag{1}
\\&= \frac{1}{2}||w||_2^2 - \sum\limits_{i=1}^{m}\alpha_i[y_i(w^Tx_i + b) - 1 + \xi_i] + \sum\limits_{i=1}^{m}\alpha_i\xi_i \tag{2}
\\& = \frac{1}{2}||w||_2^2 - \sum\limits_{i=1}^{m}\alpha_i[y_i(w^Tx_i + b) - 1] \tag{3}
\\& = \frac{1}{2}w^Tw-\sum\limits_{i=1}^{m}\alpha_iy_iw^Tx_i - \sum\limits_{i=1}^{m}\alpha_iy_ib + \sum\limits_{i=1}^{m}\alpha_i \tag{4}
\\& = \frac{1}{2}w^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i -\sum\limits_{i=1}^{m}\alpha_iy_iw^Tx_i - \sum\limits_{i=1}^{m}\alpha_iy_ib + \sum\limits_{i=1}^{m}\alpha_i \tag{5}
\\& = \frac{1}{2}w^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i - w^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i - \sum\limits_{i=1}^{m}\alpha_iy_ib + \sum\limits_{i=1}^{m}\alpha_i \tag{6}
\\& = - \frac{1}{2}w^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i - \sum\limits_{i=1}^{m}\alpha_iy_ib + \sum\limits_{i=1}^{m}\alpha_i \tag{7}
\\& = - \frac{1}{2}w^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i - b\sum\limits_{i=1}^{m}\alpha_iy_i + \sum\limits_{i=1}^{m}\alpha_i \tag{8}
\\& = -\frac{1}{2}(\sum\limits_{i=1}^{m}\alpha_iy_ix_i)^T(\sum\limits_{i=1}^{m}\alpha_iy_ix_i) - b\sum\limits_{i=1}^{m}\alpha_iy_i + \sum\limits_{i=1}^{m}\alpha_i \tag{9}
\\& = -\frac{1}{2}\sum\limits_{i=1}^{m}\alpha_iy_ix_i^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i - b\sum\limits_{i=1}^{m}\alpha_iy_i + \sum\limits_{i=1}^{m}\alpha_i \tag{10}
\\& = -\frac{1}{2}\sum\limits_{i=1}^{m}\alpha_iy_ix_i^T\sum\limits_{i=1}^{m}\alpha_iy_ix_i + \sum\limits_{i=1}^{m}\alpha_i \tag{11}
\\& = -\frac{1}{2}\sum\limits_{i=1,j=1}^{m}\alpha_iy_ix_i^T\alpha_jy_jx_j + \sum\limits_{i=1}^{m}\alpha_i \tag{12}
\\& = \sum\limits_{i=1}^{m}\alpha_i - \frac{1}{2}\sum\limits_{i=1,j=1}^{m}\alpha_i\alpha_jy_iy_jx_i^Tx_j \tag{13}
\end{align}$$

其中，(1)式到(2)式用到了$C−α_i−μ_i=0$(2)式到(3)式合并了同类项，(3)式到(4)式用到了范数的定义$||w||^2=w^Tw$(4)式到(5)式用到了上面的$w=\sum\limits_{i=1}^{m}α_iy_ix_i$(5)式到(6)式把和样本无关的$w^T$提前，(6)式到(7)式合并了同类项，(7)式到(8)式把和样本无关的b提前，(8)式到(9)式继续用到$w=\sum\limits_{i=1}^{m}α_iy_ix_i$(9）式到(10)式用到了向量的转置。由于常量的转置是其本身，所有只有向量$x_i$被转置，（10）式到(11)式用到了上面的$\sum\limits_{i=1}^{m}α_iy_i=0$（11）式到(12)式使用了$(a+b+c+…)(a+b+c+…)=aa+ab+ac+ba+bb+bc+…$的乘法运算法则，（12）式到(13)式仅仅是位置的调整。   
**拉格朗日对偶函数为：**
$$L_D = \sum\limits_{i=1}^{m}\alpha_i - \frac{1}{2}\sum\limits_{i=1,j=1}^{m}\alpha_i\alpha_jy_iy_j\Phi(x_i)\Phi(x_j)$$
$$s.t.\; C \geq \alpha_i \geq 0$$
这种状况下的拉格朗日对偶函数看起来和线性可分状况下的对偶函数一模一样，但是需要注意的是，在这个函数中，拉格朗日乘数的取值的限制改变了。在硬间隔的状况下，拉格朗日乘数值需要大于等于0，而现在它被要求不能够大于用来控制惩罚项的惩罚力度的系数C。有了对偶函数之后，我们的求解过程和硬间隔下的步骤一致。以上所有的公式，是以线性硬间隔数据为基础，考虑了软间隔存在的情况和数据是非线性的状况而得来的。而这些公式，就是sklearn类SVC背后使用的最终公式。公式中现在唯一的新变量，松弛系数的惩罚力度C，由我们的参数C来进行控制
## 重要参数C
参数C用于权衡”训练样本的正确分类“与”决策函数的边际最大化“两个不可同时完成的目标，希望找出一个平衡点来 让模型的效果最佳。
>浮点数，默认1，必须大于等于0，可不填    
>松弛系数的惩罚项系数。如果C值设定比较大，那SVC可能会选择边际较小的，能够更好地分类所有训练点的决策边界，不过模型的训练时间也会更长。如果C的设定值较小，那SVC会尽量最大化边界，决 策功能会更简单，但代价是训练的准确度。换句话说，C在SVM中的影响就像正则化参数对逻辑回归的影响。

在实际使用中，C和核函数的相关参数（gamma，degree等等）们搭配，往往是SVM调参的重点。与gamma不同，C没有在对偶函数中出现，并且是明确了调参目标的，所以我们可以明确我们究竟是否需要训练集上的高精确度来调整C的方向。默认情况下C为1，通常来说这都是一个合理的参数。 如果我们的数据很嘈杂，那我们往往减小C。当然，我们也可以使用网格搜索或者学习曲线来调整C的值。

In [ ]:
#调线性核函数
score = []
C_range = np.linspace(0.01,30,50)
for i in C_range:
    clf = SVC(kernel="linear",C=i,cache_size=5000).fit(Xtrain,Ytrain)
    score.append(clf.score(Xtest,Ytest))
print(max(score), C_range[score.index(max(score))])
plt.plot(C_range,score)
plt.show()
 
#换rbf
score = []
C_range = np.linspace(0.01,30,50)
for i in C_range:
    clf = SVC(kernel="rbf",C=i,gamma = 0.012742749857031322,cache_size=5000).fit(Xtrain,Ytrain)
    score.append(clf.score(Xtest,Ytest))
    
print(max(score), C_range[score.index(max(score))])
plt.plot(C_range,score)
plt.show()
 
#进一步细化
score = []
C_range = np.linspace(5,7,50)
for i in C_range:
    clf = SVC(kernel="rbf",C=i,gamma = 0.012742749857031322,cache_size=5000).fit(Xtrain,Ytrain)
    score.append(clf.score(Xtest,Ytest))
    
print(max(score), C_range[score.index(max(score))])
plt.plot(C_range,score)
plt.show()

此时，我们找到了乳腺癌数据集上的最优解：rbf核函数下的98.24%的准确率。当然，我们还可以使用交叉验证来 改进我们的模型，获得不同测试集和训练集上的交叉验证结果。但上述过程，为大家展现了如何选择正确的核函数，以及如何调整核函数的参数，过程虽然简单，但是希望可以对大家有所启发。
# 总结

本次向大家介绍了支持向量机的损失函数，拉格朗日函数，拉格朗日对偶函数，预测函数以及这些函数在非线性，软间隔这些情况上的推广。我们介绍了四种核函数，包括它们的特点，适合什么样的数据，有什么相关参数，优缺点，以及什么时候使用。最后我们还讲解了核函数在相关参数上的调参。  
SVM算法是一个很优秀的算法，在集成学习和神经网络之类的算法没有表现出优越性能前，SVM基本占据了分类模型的统治地位。目前则是在大数据时代的大样本背景下,SVM由于其在大样本时超级大的计算量，热度有所下降，但是仍然是一个常用的机器学习算法。

　　　　SVM算法的主要优点有：

　　　　1) 解决高维特征的分类问题和回归问题很有效,在特征维度大于样本数时依然有很好的效果。

　　　　2) 仅仅使用一部分支持向量来做超平面的决策，无需依赖全部数据。

　　　　3) 有大量的核函数可以使用，从而可以很灵活的来解决各种非线性的分类回归问题。

　　　　4)样本量不是海量数据的时候，分类准确率高，泛化能力强。

　　　　SVM算法的主要缺点有：

　　　　1) 如果特征维度远远大于样本数，则SVM表现一般。

　　　　2) SVM在样本量非常大，核函数映射维度非常高时，计算量过大，不太适合使用。

　　　　3）非线性问题的核函数的选择没有通用标准，难以选择一个合适的核函数。

　　　　4）SVM对缺失数据敏感。

# 附录
## SVC的参数列表
![svmparam1](../images/svmparam1.png)
![svmparam2](../images/svmparam2.png)
![svmparam3](../images/svmparam3.png)
## SVC的属性列表
![svmprop1](../images/svmprop1.png)
## SVC的接口列表
![svminf1](../images/svminf1.png)
![svminf1](../images/svminf2.png)
